In [1]:
import pandas as pd
import sqlalchemy as db
import cx_Oracle as orcl
import datetime as dt
import numpy as np

print("cx_Oracle.version:", orcl.version)
# show all columns without without truncating
pd.set_option('display.max_columns', None)
import sys
reload(sys)
sys.setdefaultencoding('utf8')


('cx_Oracle.version:', '7.2.3')


In [ ]:
host = "10.21.18.15"
port = "1521"
sid = "PROFITSDB"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("mobileuser","Kenya1234##", dsn)
cur = orcl.Cursor(connection) 


In [ ]:
host = "10.21.18.13"
port = "1521"
sid = "ECHODB1"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("ALLAN.ASWANI","Password1!", dsn)
cur = orcl.Cursor(connection) 


In [56]:
host="128.2.5.46"
port="1521"
sid="PROFITSDB"
username="MOBILEUSER"
password="nathan"
dsn_tns = orcl.makedsn(host, port,sid)
connection = orcl.connect(user=username, password=password, dsn=dsn_tns)
cur = orcl.Cursor(connection) 

##checking customer information
statements = """SELECT * FROM CUSTOMER"""
customer = pd.read_sql_query(statements, connection)
pd.set_option('display.max_columns', None)
customer.head(4)

###latest customer numbers
customer.tail(4)

statement_trx = """select P.ACCOUNT_NUMBER,p.cust_id,
trim(C.SURNAME) || ' '|| trim(C.FIRST_NAME) ||' ' || trim(C.MIDDLE_NAME) as name,
D.ID_TRANSACT,
d.TIMESTAMP,
D.ID_JUSTIFIC,
J.DESCRIPTION,
DECODE (D.DEBIT_CREDIT_FLAG,'2','CR','1','DR') "DR/CR",
D.ENTRY_AMOUNT,
D.PREV_ACC_BALANCE, 
D.VALUE_DATE,
D.TRX_USR,
D.ENTRY_COMMENTS
from fst_demand_extrait d, justific j, profits_account p, customer c
where
D.ID_JUSTIFIC = J.ID_JUSTIFIC
and
P.CUST_ID = C.CUST_ID
and
D.FK_DEPOSIT_ACCOACC= P.DEP_ACC_NUMBER
and p.PRFT_SYSTEM=3
and trunc(d.TIMESTAMP) >= to_date('01-06-2019','dd-mm-yyyy')
and trunc(d.TIMESTAMP) <= to_date('30-09-2019','dd-mm-yyyy')
order by D.VALUE_DATE asc"""
trxns_data = pd.read_sql_query(statement_trx, connection)
pd.set_option('display.max_columns', None)
trxns_data.head(10)


trxns_data.tail(10)

In [57]:
statement_trx_4 = '''select D.TMSTAMP,
D.TRX_DATE,
d.TRX_UNIT,
d.TRX_USR,
d.TRX_USR_SN,
d.TUN_INTERNAL_SN,
trim(to_char(D.TRX_DATE,'yyyymmdd'))||''||trim(D.TRX_UNIT)||''||trim(D.TRX_USR)||''||trim (abs(D.TRX_USR_SN) -1)||''||trim(D.TUN_INTERNAL_SN) as TRANSACTION_REF,
P.ID_PRODUCT,
P.DESCRIPTION AS PRODUCT_DESCRIPTION,
R.ACCOUNT_NUMBER, 
A.FK_CUSTOMERCUST_ID,
C.ID_NO AS NATIONAL_ID,
S.MOBILE_TEL,
A.DESIGNATION AS ACCOUNT_NAME,
D.I_ID_JUSTIFIC,
J.DESCRIPTION AS TRANSACTION_TYPE,
ABS (D.I_AMOUNT) "TRANSACTION_AMOUNT",
D.ACC_AMOUNT_40 AS COMMISSION,
D.ACC_AMOUNT_42 AS "EXCISE DUTY",
D.O_FINAL_ACC_AMOUNT AS TOTAL_TRX_AMOUNT,
D.I_COMMENTS TRX_COMMENTS,
F.REVERSE_FLAG, 
F.REVERSED_TRX_FLAG

from prod.DEP_TRX_RECORDING D, prod.JUSTIFIC J, prod.PRODUCT P, prod.DEPOSIT_ACCOUNT A, prod.PROFITS_ACCOUNT R, prod.FST_DEMAND_EXTRAIT F, prod.OTHER_ID C, prod.CUSTOMER S 
WHERE 
A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = F.FK_DEPOSIT_ACCOACC
and A.ACCOUNT_NUMBER = F.FK_DEPOSIT_ACCOACC
AND C.FK_CUSTOMERCUST_ID=A.FK_CUSTOMERCUST_ID
AND D.ID_PRODUCT = P.ID_PRODUCT
AND D.I_ID_JUSTIFIC = J.ID_JUSTIFIC
AND A.FK_CUSTOMERCUST_ID=S.CUST_ID
AND F.TRX_DATE = D.TRX_DATE   
AND F.TRX_UNIT  = D.TRX_UNIT 
AND F.TRX_SN = D.TRX_USR_SN 
AND F.TRX_USR = D.TRX_USR
and F.TUN_INTERNAL_SN = D.TUN_INTERNAL_SN 
AND TRUNC(D.TMSTAMP) between TO_DATE('01-01-2019', 'dd-mm-yyyy') and 
TO_DATE('25-12-2019', 'dd-mm-yyyy') 
--AND D.I_ID_JUSTIFIC IN (31804,31805,31815,32096,32097,32099,32096,32097,32099) 
AND R.PRFT_SYSTEM = 3
AND R.PRODUCT_ID NOT IN ('36000','35000','34000')
AND F.REVERSE_FLAG =0
AND F.REVERSED_TRX_FLAG=0
'''
trxns_data_trx_4 = pd.read_sql_query(statement_trx_4, connection)
trxns_data_trx_4

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,ID_PRODUCT,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,EXCISE DUTY,TOTAL_TRX_AMOUNT,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,2019-12-06 16:16:12,2019-12-06,100,MIPU0001,2164,1,20191206100MIPU000121631,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,16001,CHEQUE DEPOSIT OF OTHER BANK (ELEC.CLER),33233.00,0.0,0.0,33233.00,HOUSING FINANCE-CPF LASERAND STANCHART,0,0
1,2019-11-29 09:07:39,2019-11-29,100,ENM1227,194,1,20191129100ENM12271931,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,33100,JOURNAL CREDIT,9468.80,0.0,0.0,9468.80,BANK CHARGES WRITE-OFF,0,0
2,2019-11-29 09:06:25,2019-11-29,100,ENM1227,38,1,20191129100ENM1227371,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,33100,JOURNAL CREDIT,12864.76,0.0,0.0,12864.76,BANK CHARGES WRITE-OFF,0,0
3,2019-11-15 11:21:17,2019-11-15,100,ENM1227,5890,1,20191115100ENM122758891,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,4764.78,0.0,0.0,4764.78,0600067009-CLAIM not found.,0,0
4,2019-11-15 11:20:51,2019-11-15,100,ENM1227,5809,1,20191115100ENM122758081,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,4621.77,0.0,0.0,4621.77,0600067572-CLAIM not found.,0,0
5,2019-11-15 11:20:45,2019-11-15,100,ENM1227,5793,1,20191115100ENM122757921,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,1026.61,0.0,0.0,1026.61,0600095577-CLAIM not found.,0,0
6,2019-11-15 11:20:15,2019-11-15,100,ENM1227,5694,1,20191115100ENM122756931,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,5498.53,0.0,0.0,5498.53,0600095305-CLAIM not found.,0,0
7,2019-11-15 11:19:35,2019-11-15,100,ENM1227,5570,1,20191115100ENM122755691,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,1038.59,0.0,0.0,1038.59,0600048552-CLAIM not found.,0,0
8,2019-11-15 11:19:22,2019-11-15,100,ENM1227,5528,1,20191115100ENM122755271,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,3197.74,0.0,0.0,3197.74,0600002003-CLAIM not found.,0,0
9,2019-11-14 14:14:03,2019-11-14,100,ENM1227,19758,1,20191114100ENM1227197571,32000,MEMO PRODUCT-JOURNAL,0000430004,43,00000,,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,31639,SALARY POSTINGS RETURN REJECTED AMOUNTS,23927.58,0.0,0.0,23927.58,0600127107-CLAIM not found.,0,0


In [3]:
trxns_data_trx_4 = pd.read_csv("/Users/allan.aswani/Downloads/calculation/transactions_2019_11_25.csv")

In [4]:
trxns_data_trx_4

,Unnamed: 0,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,0,2019-11-07 08:42,2019-11-07,17,WML2084,4,1,2019110717WML208431,HFC SALARY ACCOUNT,2301083201,31501,230108,6431666,2.54716E+11,PETER IRUNGU KAMAU ...,31615,OTC CASH WITHDRAWAL,8550.0,100.0,,0,0
1,1,2019-03-06 15:01,2019-03-06,20,OMN1986,112,1,2019030620OMN19861111,HFC SALARY ACCOUNT,6648883201,31501,664888,21991533,2.54729E+11,CATHERINE WANGARE NJUGUNA ...,31615,OTC CASH WITHDRAWAL,9700.0,100.0,,0,0
2,2,2019-04-05 21:27,2019-04-06,109,KOCE0003,314,1,20190406109KOCE00033131,VIRTUAL ACCOUNT MOBILE,6686233201,31572,668623,21257746,2.54713E+11,CATHERINE SAANI ...,32099,ACCOUNT TO MPESA(B2C),2650.0,54.0,SEND TO MPESA 254712943509,0,0
3,3,2019-10-22 12:32,2019-10-22,19,PWK1613,75,1,2019102219PWK1613741,TRANSACTIONAL ACCOUNT-CA,2000000323,31719,857,10899859,2.54722E+11,CATHERINE GAKII KIBITI ...,31615,OTC CASH WITHDRAWAL,20000.0,100.0,,0,0
4,4,2019-05-30 17:38,2019-05-30,109,KOCE0002,4191,1,20190530109KOCE000241901,YOLO ACCOUNT,6400643201,31521,640064,36674185,2.54725E+11,OLIVER KIBET CHEREN ...,32099,ACCOUNT TO MPESA(B2C),300.0,23.0,SEND TO MPESA 254724672405,0,0
5,5,2019-09-08 18:33,2019-09-09,109,KOCE0001,4571,1,20190909109KOCE000145701,TRANSACTIONAL ACCOUNT-CA,7190010573,31719,8809,8925590,2.54734E+11,JAMES WILSON OMOLO JAGUGA ...,32099,ACCOUNT TO MPESA(B2C),1000.0,30.0,SEND TO MPESA 254725335540,0,0
6,6,2019-10-08 20:44,2019-10-09,109,KOCE0003,26,1,20191009109KOCE0003251,PERSONAL CURRENT ACCOUNT-UNBUN,2227773201,31718,222777,34083225,2.54729E+11,MICHELLE MWARANTA G GICHUHI ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254729419006,0,0
7,7,2019-07-26 09:59,2019-07-26,280,GNG2042,5,1,20190726280GNG204241,TRANSACTIONAL ACCOUNT-CA,1954703201,31719,195470,1901824,2.54721E+11,PROF. PETER MACHATHA GITU ...,31615,OTC CASH WITHDRAWAL,50000.0,100.0,,0,0
8,8,2019-05-04 06:14,2019-05-04,230,10010002,3,1,2.01905E+20,HFC SALARY ACCOUNT,1474223203,31501,147422,22403150,2.54716E+11,JUSTUS OYARO ORAYO ...,33000,ATM WITHDRAWAL ONUS KENSWITCH,5000.0,35.0,ATM DEBIT,0,0
9,9,2019-03-20 09:43,2019-03-20,109,KOCE0001,10905,1,20190320109KOCE0001109041,VIRTUAL ACCOUNT MOBILE,6006563201,31572,600656,34758978,2.54742E+11,BENJAMIN MUKANDA ...,32099,ACCOUNT TO MPESA(B2C),450.0,23.0,SEND TO MPESA 254741635119,0,0


In [5]:
df1 = trxns_data_trx_4
df1

,Unnamed: 0,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
0,0,2019-11-07 08:42,2019-11-07,17,WML2084,4,1,2019110717WML208431,HFC SALARY ACCOUNT,2301083201,31501,230108,6431666,2.54716E+11,PETER IRUNGU KAMAU ...,31615,OTC CASH WITHDRAWAL,8550.0,100.0,,0,0
1,1,2019-03-06 15:01,2019-03-06,20,OMN1986,112,1,2019030620OMN19861111,HFC SALARY ACCOUNT,6648883201,31501,664888,21991533,2.54729E+11,CATHERINE WANGARE NJUGUNA ...,31615,OTC CASH WITHDRAWAL,9700.0,100.0,,0,0
2,2,2019-04-05 21:27,2019-04-06,109,KOCE0003,314,1,20190406109KOCE00033131,VIRTUAL ACCOUNT MOBILE,6686233201,31572,668623,21257746,2.54713E+11,CATHERINE SAANI ...,32099,ACCOUNT TO MPESA(B2C),2650.0,54.0,SEND TO MPESA 254712943509,0,0
3,3,2019-10-22 12:32,2019-10-22,19,PWK1613,75,1,2019102219PWK1613741,TRANSACTIONAL ACCOUNT-CA,2000000323,31719,857,10899859,2.54722E+11,CATHERINE GAKII KIBITI ...,31615,OTC CASH WITHDRAWAL,20000.0,100.0,,0,0
4,4,2019-05-30 17:38,2019-05-30,109,KOCE0002,4191,1,20190530109KOCE000241901,YOLO ACCOUNT,6400643201,31521,640064,36674185,2.54725E+11,OLIVER KIBET CHEREN ...,32099,ACCOUNT TO MPESA(B2C),300.0,23.0,SEND TO MPESA 254724672405,0,0
5,5,2019-09-08 18:33,2019-09-09,109,KOCE0001,4571,1,20190909109KOCE000145701,TRANSACTIONAL ACCOUNT-CA,7190010573,31719,8809,8925590,2.54734E+11,JAMES WILSON OMOLO JAGUGA ...,32099,ACCOUNT TO MPESA(B2C),1000.0,30.0,SEND TO MPESA 254725335540,0,0
6,6,2019-10-08 20:44,2019-10-09,109,KOCE0003,26,1,20191009109KOCE0003251,PERSONAL CURRENT ACCOUNT-UNBUN,2227773201,31718,222777,34083225,2.54729E+11,MICHELLE MWARANTA G GICHUHI ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254729419006,0,0
7,7,2019-07-26 09:59,2019-07-26,280,GNG2042,5,1,20190726280GNG204241,TRANSACTIONAL ACCOUNT-CA,1954703201,31719,195470,1901824,2.54721E+11,PROF. PETER MACHATHA GITU ...,31615,OTC CASH WITHDRAWAL,50000.0,100.0,,0,0
8,8,2019-05-04 06:14,2019-05-04,230,10010002,3,1,2.01905E+20,HFC SALARY ACCOUNT,1474223203,31501,147422,22403150,2.54716E+11,JUSTUS OYARO ORAYO ...,33000,ATM WITHDRAWAL ONUS KENSWITCH,5000.0,35.0,ATM DEBIT,0,0
9,9,2019-03-20 09:43,2019-03-20,109,KOCE0001,10905,1,20190320109KOCE0001109041,VIRTUAL ACCOUNT MOBILE,6006563201,31572,600656,34758978,2.54742E+11,BENJAMIN MUKANDA ...,32099,ACCOUNT TO MPESA(B2C),450.0,23.0,SEND TO MPESA 254741635119,0,0


In [6]:
df1.ACCOUNT_NUMBER = df1.ACCOUNT_NUMBER.astype('int64')

In [7]:
account_100 = df1.query('ACCOUNT_NUMBER == "9340593201"')
account_100

,Unnamed: 0,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
10627,10627,2019-07-14 10:20,2019-07-15,109,KOCE0001,4830,1,20190715109KOCE000148291,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254700294591,0,0
20760,20760,2019-07-22 09:09,2019-07-22,109,KOCE0001,14356,1,20190722109KOCE0001143551,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254700294591,0,0
33109,33109,2019-08-01 00:44,2019-08-01,109,KOCE0001,973,1,20190801109KOCE00019721,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),25000.0,60.0,SEND TO MPESA 254720211892,0,0
146760,146760,2019-07-30 16:00,2019-07-30,109,KOCE0001,11040,1,20190730109KOCE0001110391,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),20000.0,60.0,SEND TO MPESA 254722141284,0,0
254437,254437,2019-07-07 11:41,2019-07-08,109,KOCE0001,5977,1,20190708109KOCE000159761,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),5000.0,60.0,SEND TO MPESA 254700294591,0,0
264489,264489,2019-07-27 11:35,2019-07-27,109,KOCE0001,7061,1,20190727109KOCE000170601,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),61800.0,60.0,SEND TO MPESA 254722141284,0,0
358030,358030,2019-08-01 07:33,2019-08-01,109,KOCE0001,2171,1,20190801109KOCE000121701,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),23000.0,60.0,SEND TO MPESA 254722141284,0,0
360537,360537,2019-07-15 19:25,2019-07-15,109,KOCE0001,23935,1,20190715109KOCE0001239341,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),10000.0,60.0,SEND TO MPESA 254700294591,0,0
385565,385565,2019-08-25 14:00,2019-08-26,109,KOCE0001,7274,1,20190826109KOCE000172731,CURRENT ACCOUNT STAFF,9340593201,31705,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),51000.0,22.0,SEND TO MPESA 254722141284,0,0
472698,472698,2019-07-30 11:31,2019-07-30,109,KOCE0001,6426,1,20190730109KOCE000164251,VIRTUAL ACCOUNT MOBILE,9340593201,31572,934059,22844723,2.54722E+11,OKERO JOHN NATHAN ...,32099,ACCOUNT TO MPESA(B2C),5000.0,60.0,SEND TO MPESA 254722141284,0,0


In [8]:
#column names and type
df1.dtypes.to_list

<bound method Series.tolist of Unnamed: 0               int64
TMSTAMP                 object
TRX_DATE                object
TRX_UNIT                 int64
TRX_USR                 object
TRX_USR_SN               int64
TUN_INTERNAL_SN          int64
TRANSACTION_REF         object
PRODUCT_DESCRIPTION     object
ACCOUNT_NUMBER           int64
ID_PRODUCT               int64
FK_CUSTOMERCUST_ID       int64
NATIONAL_ID             object
MOBILE_TEL              object
ACCOUNT_NAME            object
I_ID_JUSTIFIC            int64
TRANSACTION_TYPE        object
TRANSACTION_AMOUNT     float64
COMMISSION             float64
TRX_COMMENTS            object
REVERSE_FLAG             int64
REVERSED_TRX_FLAG        int64
dtype: object>

In [12]:
df2 = df1.query('ID_PRODUCT == "31111"' or 'ID_PRODUCT == "31705"' or 'ID_PRODUCT == "31200"')
df2.sort_values(ascending=False,by='TMSTAMP')
df2

,Unnamed: 0,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
27,27,2019-06-15 13:56,2019-06-15,109,KOCE0001,10129,1,20190615109KOCE0001101281,STAFF SALARY ACCOUNT,2000056439,31111,87600,22442201,2.54721E+11,AMBROSE WARUTERE NGINGA ...,32099,ACCOUNT TO MPESA(B2C),2027.0,22.0,SEND TO MPESA 254714714383,0,0
102,102,2019-07-03 09:58,2019-07-03,109,KOCE0001,5954,1,20190703109KOCE000159531,STAFF SALARY ACCOUNT,2000033156,31111,38623,13224078,2.54729E+11,JAMES MUMINA MUENDO ...,32099,ACCOUNT TO MPESA(B2C),5000.0,22.0,SEND TO MPESA 254720895962,0,0
105,105,2019-05-30 18:25,2019-05-30,109,KOCE0002,4491,1,20190530109KOCE000244901,STAFF SALARY ACCOUNT,2000107436,31111,172986,22506793,2.54712E+11,CYPRIAN MUYESHI ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254711581765,0,0
122,122,2019-07-27 15:02,2019-07-27,109,KOCE0002,2029,1,20190727109KOCE000220281,STAFF SALARY ACCOUNT,2000115001,31111,188319,26130176,2.54729E+11,KIMWETICH KIPRUTO KELVIN ...,32099,ACCOUNT TO MPESA(B2C),2500.0,22.0,SEND TO MPESA 254728814964,0,0
180,180,2019-05-29 17:40,2019-05-29,109,KOCE0003,759,1,20190529109KOCE00037581,STAFF SALARY ACCOUNT,8000543101,31111,800054,25768523,2.5473E+11,EVANS OTIENO ONGODO ...,32099,ACCOUNT TO MPESA(B2C),3000.0,22.0,SEND TO MPESA 254729505957,0,0
197,197,2019-05-27 18:15,2019-05-27,109,KOCE0001,30325,1,20190527109KOCE0001303241,STAFF SALARY ACCOUNT,2000077866,31111,113304,23475860,2.54725E+11,SYLVESTER MUTUA KIVINDYO ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254724811282,0,0
211,211,2019-07-30 13:20,2019-07-30,109,KOCE0001,8324,1,20190730109KOCE000183231,STAFF SALARY ACCOUNT,7201923201,31111,720192,22197290,2.54722E+11,ROSE MUTURI ...,32099,ACCOUNT TO MPESA(B2C),12500.0,22.0,SEND TO MPESA 0720000748,0,0
309,309,2019-05-11 20:11,2019-05-13,109,KOCE0003,50,1,20190513109KOCE0003491,STAFF SALARY ACCOUNT,2000033012,31111,41545,11606278,2.54722E+11,GEOFREY MWAURA NGARI ...,32099,ACCOUNT TO MPESA(B2C),3000.0,22.0,SEND TO MPESA 254720982300,0,0
378,378,2019-06-28 00:25,2019-06-28,109,KOCE0001,2143,1,20190628109KOCE000121421,STAFF SALARY ACCOUNT,3095533101,31111,309553,23298892,2.54726E+11,PATRICIA KENDELI SALANO ...,32099,ACCOUNT TO MPESA(B2C),4500.0,22.0,SEND TO MPESA 254722279351,0,0
512,512,2019-07-05 21:51,2019-07-06,109,KOCE0002,397,1,20190706109KOCE00023961,STAFF SALARY ACCOUNT,9022143201,31111,902214,28846367,2.54715E+11,ALEX KASWII ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254714964847,0,0


In [1]:
#staff accounts 
df2 = df1.query('ID_PRODUCT == "31111"' or 'ID_PRODUCT == "31705"' or 'ID_PRODUCT == "31200"') 

df2.sort_values(ascending=False,by='TMSTAMP')
df2

NameError: name 'df1' is not defined

In [11]:
#df1['TRX_USR'].unique().tolist()

In [12]:
#staff accounts 
#df_test = df1.query('PRODUCT_DESCRIPTION == "STAFF SALARY ACCOUNT"') 
#df2.TMSTAMP.sort_values(ascending = False)

In [13]:
df3 = df2.query('TRX_USR.str.contains("KOCE")' or 'TRX_USR.str.contains("ESB")')
df3

,Unnamed: 0,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG
27,27,2019-06-15 13:56,2019-06-15,109,KOCE0001,10129,1,20190615109KOCE0001101281,STAFF SALARY ACCOUNT,2000056439,31111,87600,22442201,2.54721E+11,AMBROSE WARUTERE NGINGA ...,32099,ACCOUNT TO MPESA(B2C),2027.0,22.0,SEND TO MPESA 254714714383,0,0
102,102,2019-07-03 09:58,2019-07-03,109,KOCE0001,5954,1,20190703109KOCE000159531,STAFF SALARY ACCOUNT,2000033156,31111,38623,13224078,2.54729E+11,JAMES MUMINA MUENDO ...,32099,ACCOUNT TO MPESA(B2C),5000.0,22.0,SEND TO MPESA 254720895962,0,0
105,105,2019-05-30 18:25,2019-05-30,109,KOCE0002,4491,1,20190530109KOCE000244901,STAFF SALARY ACCOUNT,2000107436,31111,172986,22506793,2.54712E+11,CYPRIAN MUYESHI ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254711581765,0,0
122,122,2019-07-27 15:02,2019-07-27,109,KOCE0002,2029,1,20190727109KOCE000220281,STAFF SALARY ACCOUNT,2000115001,31111,188319,26130176,2.54729E+11,KIMWETICH KIPRUTO KELVIN ...,32099,ACCOUNT TO MPESA(B2C),2500.0,22.0,SEND TO MPESA 254728814964,0,0
180,180,2019-05-29 17:40,2019-05-29,109,KOCE0003,759,1,20190529109KOCE00037581,STAFF SALARY ACCOUNT,8000543101,31111,800054,25768523,2.5473E+11,EVANS OTIENO ONGODO ...,32099,ACCOUNT TO MPESA(B2C),3000.0,22.0,SEND TO MPESA 254729505957,0,0
197,197,2019-05-27 18:15,2019-05-27,109,KOCE0001,30325,1,20190527109KOCE0001303241,STAFF SALARY ACCOUNT,2000077866,31111,113304,23475860,2.54725E+11,SYLVESTER MUTUA KIVINDYO ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254724811282,0,0
211,211,2019-07-30 13:20,2019-07-30,109,KOCE0001,8324,1,20190730109KOCE000183231,STAFF SALARY ACCOUNT,7201923201,31111,720192,22197290,2.54722E+11,ROSE MUTURI ...,32099,ACCOUNT TO MPESA(B2C),12500.0,22.0,SEND TO MPESA 0720000748,0,0
309,309,2019-05-11 20:11,2019-05-13,109,KOCE0003,50,1,20190513109KOCE0003491,STAFF SALARY ACCOUNT,2000033012,31111,41545,11606278,2.54722E+11,GEOFREY MWAURA NGARI ...,32099,ACCOUNT TO MPESA(B2C),3000.0,22.0,SEND TO MPESA 254720982300,0,0
378,378,2019-06-28 00:25,2019-06-28,109,KOCE0001,2143,1,20190628109KOCE000121421,STAFF SALARY ACCOUNT,3095533101,31111,309553,23298892,2.54726E+11,PATRICIA KENDELI SALANO ...,32099,ACCOUNT TO MPESA(B2C),4500.0,22.0,SEND TO MPESA 254722279351,0,0
512,512,2019-07-05 21:51,2019-07-06,109,KOCE0002,397,1,20190706109KOCE00023961,STAFF SALARY ACCOUNT,9022143201,31111,902214,28846367,2.54715E+11,ALEX KASWII ...,32099,ACCOUNT TO MPESA(B2C),1000.0,15.0,SEND TO MPESA 254714964847,0,0


In [30]:
#period of running 
#df3.TRX_DATE#.max()

#df2.TRX_DATE =  pd.to_datetime(df2.TRX_DATE, infer_datetime_format=True)
#df3.TMSTAMP.sort_values(ascending=False)
#df3
df3.TRX_DATE

27        2019-06-15
102       2019-07-03
105       2019-05-30
122       2019-07-27
180       2019-05-29
197       2019-05-27
211       2019-07-30
309       2019-05-13
378       2019-06-28
512       2019-07-06
832       2019-08-03
862       2019-08-30
875       2019-05-22
1035      2019-07-30
1077      2019-07-02
1124      2019-08-07
1307      2019-08-28
1535      2019-05-28
1808      2019-06-26
1942      2019-08-10
2125      2019-05-30
2518      2019-06-06
2553      2019-06-22
2618      2019-07-15
2683      2019-06-03
2703      2019-05-27
2718      2019-06-15
2867      2019-07-02
3023      2019-08-24
3116      2019-06-14
             ...    
948983    2019-05-16
949425    2019-08-19
949458    2019-07-01
949557    2019-05-15
949619    2019-06-29
949624    2019-08-03
949649    2019-05-27
949702    2019-08-26
949847    2019-07-08
950222    2019-05-29
950564    2019-05-20
950841    2019-08-15
951045    2019-07-29
951072    2019-06-10
951078    2019-07-05
951495    2019-06-28
951579    201

In [19]:
#df3.dtypes
NOW = df3.TRX_DATE.max()
df3.groupby('ACCOUNT_NUMBER').a
#df3.groupby('ACCOUNT_NUMBER').agg({'TRX_DATE': lambda x: (NOW - x.max()).days})#.agg({'TRX_DATE': lambda x: (NOW - x.max()).days})#, 'TRANSACTION_REF': lambda x: len(x), 'COMMISSION': lambda x: x.sum()})

In [20]:
df3.columns

Index([u'Unnamed: 0', u'TMSTAMP', u'TRX_DATE', u'TRX_UNIT', u'TRX_USR',
       u'TRX_USR_SN', u'TUN_INTERNAL_SN', u'TRANSACTION_REF',
       u'PRODUCT_DESCRIPTION', u'ACCOUNT_NUMBER', u'ID_PRODUCT',
       u'FK_CUSTOMERCUST_ID', u'NATIONAL_ID', u'MOBILE_TEL', u'ACCOUNT_NAME',
       u'I_ID_JUSTIFIC', u'TRANSACTION_TYPE', u'TRANSACTION_AMOUNT',
       u'COMMISSION', u'TRX_COMMENTS', u'REVERSE_FLAG', u'REVERSED_TRX_FLAG'],
      dtype='object')

In [15]:
NOW = df3.TRX_DATE.max()
rfmTable = df3.groupby('ACCOUNT_NUMBER').agg({'TRX_DATE': lambda x: (NOW - x.max()).days, 'TRANSACTION_REF': lambda x: len(x), 'COMMISSION': lambda x: x.sum()})
rfmTable['TRX_DATE'] = rfmTable['TRX_DATE'].astype(int)
rfmTable.rename(columns={'TRX_DATE': 'recency', 
                         'TRANSACTION_REF': 'frequency', 
                         'COMMISSION': 'monetary_value'}, inplace=True)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [70]:
rfmTable

,frequency,recency,monetary_value
ACCOUNT_NUMBER,,,
201643101,12,25,0.0
949563101,791,0,4731.0
1098683102,750,2,5640.0
1302963102,155,179,555.0
1445193102,1,119,0.0
1591173102,3,133,66.0
1602073103,1,94,0.0
1619473101,1,213,0.0
1637003102,44,25,486.0


In [71]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
print(quantiles)
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 24.0, 0.5: 90.0, 0.75: 266.5},
 'monetary_value': {0.25: 0.0, 0.5: 507.0, 0.75: 1795.5},
 'recency': {0.25: 0.0, 0.5: 4.0, 0.75: 27.5}}

In [72]:
#reference https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6
segmented_rfm = rfmTable

In [73]:
#The lowest recency, highest frequency and monetary amounts are our best customers.
def FMScore(x,p,d):   
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def RScore(x,p,d):   
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [74]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()


,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile
ACCOUNT_NUMBER,,,,,,
201643101,12,25,0.0,2,1,1
949563101,791,0,4731.0,4,4,4
1098683102,750,2,5640.0,3,4,4
1302963102,155,179,555.0,1,3,3
1445193102,1,119,0.0,1,1,1


In [75]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str)   + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
ACCOUNT_NUMBER,,,,,,,
201643101,12,25,0.0,2,1,1,211
949563101,791,0,4731.0,4,4,4,444
1098683102,750,2,5640.0,3,4,4,344
1302963102,155,179,555.0,1,3,3,133
1445193102,1,119,0.0,1,1,1,111


In [76]:
segmented_rfm.RFMScore.max()

'444'

In [77]:
staff_identify = pd.read_excel (r'/Users/allan.aswani/Documents/scorecardworks/staff_loans_version_1.xlsx', sheet_name='Sheet2')
staff_identify

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,National_id,whizz_phone
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,8515233,254722310293
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,9778506,254722466794
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6141908,254722338270
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,10874594,254722559933
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,9328141,254722859902
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,10314300,254721249078
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,10269408,254720269385
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,10880731,254722633539
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,8182751,254714606788
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,10087038,254722790605


In [78]:
segmented_rfm = pd.DataFrame(segmented_rfm)
segmented_rfm.reset_index(level=0, inplace=True)
segmented_rfm

,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,201643101,12,25,0.0,2,1,1,211
1,949563101,791,0,4731.0,4,4,4,444
2,1098683102,750,2,5640.0,3,4,4,344
3,1302963102,155,179,555.0,1,3,3,133
4,1445193102,1,119,0.0,1,1,1,111
5,1591173102,3,133,66.0,1,1,2,112
6,1602073103,1,94,0.0,1,1,1,111
7,1619473101,1,213,0.0,1,1,1,111
8,1637003102,44,25,486.0,2,2,2,222
9,1687573101,2,17,0.0,2,1,1,211


In [79]:
staff_identify.dtypes

Pfno                             object
Name                             object
ACCOUNT                          object
MOBILE                           object
Loan Limit                        int64
Type of whizz Account            object
cust_id                         float64
target saving                   float64
fixed saving                    float64
opening date             datetime64[ns]
grade                            object
income proxy                    float64
accountNumber                   float64
cbsCustomerID                   float64
customerID                      float64
customerNames                    object
idNumber                         object
mobileNumber                    float64
productCode                     float64
National_id                      object
whizz_phone                       int64
dtype: object

In [80]:
print(segmented_rfm)

In [81]:
staff_account_limit = staff_identify.merge(segmented_rfm,left_on='ACCOUNT', right_on="ACCOUNT_NUMBER", how = "left")

In [82]:
staff_identify

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,National_id,whizz_phone
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,8515233,254722310293
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,9778506,254722466794
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6141908,254722338270
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,10874594,254722559933
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,9328141,254722859902
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,10314300,254721249078
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,10269408,254720269385
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,10880731,254722633539
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,8182751,254714606788
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,10087038,254722790605


In [83]:
staff_account_limit

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,National_id,whizz_phone,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,8515233,254722310293,2.000033e+09,158.0,4.0,1716.0,3.0,3.0,3.0,333
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,9778506,254722466794,2.000033e+09,26.0,3.0,0.0,3.0,2.0,1.0,321
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6141908,254722338270,2.000035e+09,1.0,258.0,0.0,1.0,1.0,1.0,111
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,10874594,254722559933,2.000033e+09,245.0,1.0,1611.0,3.0,3.0,3.0,333
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,9328141,254722859902,2.000033e+09,471.0,0.0,2841.0,4.0,4.0,4.0,444
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,10314300,254721249078,2.000033e+09,148.0,0.0,1083.0,4.0,3.0,3.0,433
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,10269408,254720269385,2.000033e+09,825.0,0.0,6642.0,4.0,4.0,4.0,444
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,10880731,254722633539,2.000033e+09,521.0,1.0,2232.0,3.0,4.0,4.0,344
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,8182751,254714606788,2.000033e+09,1052.0,0.0,3840.0,4.0,4.0,4.0,444
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,10087038,254722790605,2.000033e+09,225.0,0.0,1590.0,4.0,3.0,3.0,433


In [84]:
staff_account_limit.query('RFMScore>0')

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,National_id,whizz_phone,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,363,Lucy Wangui Ndungu,2000032981,722310293,6000,STAFF SALARY ACCOUNT,29624.0,0.0,0.0,2000-12-06,01,10000.0,2.000033e+09,29624.0,519577.0,LUCY WANGUI,8515233,2.547223e+11,31111.0,8515233,254722310293,2.000033e+09,158.0,4.0,1716.0,3.0,3.0,3.0,333
1,390,Rospa Jepngetich Kandie,2000033147,722466794,6000,STAFF SALARY ACCOUNT,30258.0,0.0,0.0,2018-06-13,UNC,NaN,2.000033e+09,30258.0,544942.0,ROSPA JEPNG'ETICH,9778506,2.547225e+11,31111.0,9778506,254722466794,2.000033e+09,26.0,3.0,0.0,3.0,2.0,1.0,321
2,403,Alice Wanjiru Weru,2000035405,254722338270,6000,STAFF SALARY ACCOUNT,9582.0,0.0,0.0,2001-10-04,02,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6141908,254722338270,2.000035e+09,1.0,258.0,0.0,1.0,1.0,1.0,111
3,415,George Mburu Karanu,2000032902,722559933,18000,STAFF SALARY ACCOUNT,4496.0,0.0,0.0,2000-11-29,M3,30000.0,2.000033e+09,4496.0,92.0,GEORGE MBURU,10874594,2.547226e+11,31111.0,10874594,254722559933,2.000033e+09,245.0,1.0,1611.0,3.0,3.0,3.0,333
4,436,Eric Kiptoo Kosgei,2000033074,722859902,6000,STAFF SALARY ACCOUNT,1440.0,0.0,0.0,2012-11-17,UNC,NaN,2.000033e+09,1440.0,128.0,KOSGEI ERIC,9328141,2.547229e+11,31111.0,9328141,254722859902,2.000033e+09,471.0,0.0,2841.0,4.0,4.0,4.0,444
5,439,David Kimutai Kirui,2000033024,721249078,6000,STAFF SALARY ACCOUNT,21135.0,0.0,0.0,2000-12-06,02,10000.0,2.000033e+09,21135.0,1278.0,DAVID KIRUI,10314300,2.547212e+11,31111.0,10314300,254721249078,2.000033e+09,148.0,0.0,1083.0,4.0,3.0,3.0,433
6,441,Esther Wambui Muya,2000033121,720269385,6000,STAFF SALARY ACCOUNT,21123.0,0.0,0.0,2016-08-12,UNC,NaN,2.000033e+09,21123.0,2276.0,ESTHER MUYA,10269408,2.547203e+11,31111.0,10269408,254720269385,2.000033e+09,825.0,0.0,6642.0,4.0,4.0,4.0,444
7,450,Jane Wambui Kuria,2000032960,722633539,12000,STAFF SALARY ACCOUNT,21439.0,0.0,0.0,2000-12-05,M2,20000.0,2.000033e+09,21439.0,181.0,JANE KURIA,10880731,2.547226e+11,31111.0,10880731,254722633539,2.000033e+09,521.0,1.0,2232.0,3.0,4.0,4.0,344
8,462,Gordon Mbigo Nyagudi,2000033031,714606788,18000,STAFF SALARY ACCOUNT,29488.0,0.0,0.0,2014-10-28,M3,30000.0,2.000033e+09,29488.0,29.0,GORDON NYAGUDI,8182751,2.547146e+11,31111.0,8182751,254714606788,2.000033e+09,1052.0,0.0,3840.0,4.0,4.0,4.0,444
9,463,Lucy Njoki Njogu,2000033205,722790605,12000,STAFF SALARY ACCOUNT,20997.0,0.0,0.0,2000-12-13,M2,20000.0,2.000033e+09,20997.0,136.0,LUCY NJOGU,10087038,2.547228e+11,31111.0,10087038,254722790605,2.000033e+09,225.0,0.0,1590.0,4.0,3.0,3.0,433


In [85]:
def limitAdjuster(x):
    if x in ('111','112','122','133','134','144'):
        return 1.25
    elif x in ('211','221','222','232','233','243','244'):
        return 1.5
    elif x in ('311','321','322','333','343','344'): 
        return 1.75
    elif x in ('422','433','444'): 
        return 2
    else:
        return 1

In [86]:
#,'111','112','122','133','134','144','211','221','222','232','233','243','244','311','321','322','333','343','344','422','433','444'

In [87]:
staff_account_limit['Loan Limit'] = staff_account_limit['Loan Limit'].astype('int64')

In [88]:

#staff_account_limit.RFMScore.unique #staff_account_limit.RFMScore.fillna(0, inplace = True) 

In [89]:
#staff_account_limit.RFMScore.unique

In [90]:
staff_account_limit['adjuster'] = staff_account_limit.RFMScore.apply(limitAdjuster)

In [91]:
staff_account_limit["new_limit_25_Dec"] = staff_account_limit['Loan Limit'] *staff_account_limit['adjuster']

In [92]:
staff_account_limit.dtypes

Pfno                             object
Name                             object
ACCOUNT                          object
MOBILE                           object
Loan Limit                        int64
Type of whizz Account            object
cust_id                         float64
target saving                   float64
fixed saving                    float64
opening date             datetime64[ns]
grade                            object
income proxy                    float64
accountNumber                   float64
cbsCustomerID                   float64
customerID                      float64
customerNames                    object
idNumber                         object
mobileNumber                    float64
productCode                     float64
National_id                      object
whizz_phone                       int64
ACCOUNT_NUMBER                  float64
frequency                       float64
recency                         float64
monetary_value                  float64


In [93]:
staff_account_limit.head()
staff_account_limit_nov_2019 = staff_account_limit[["Pfno", "Name","ACCOUNT","Loan Limit" ,"whizz_phone", "new_limit_25_Dec"]]

In [94]:
staff_account_limit_nov_2019

,Pfno,Name,ACCOUNT,Loan Limit,whizz_phone,new_limit_25_Dec
0,363,Lucy Wangui Ndungu,2000032981,6000,254722310293,10500.0
1,390,Rospa Jepngetich Kandie,2000033147,6000,254722466794,10500.0
2,403,Alice Wanjiru Weru,2000035405,6000,254722338270,7500.0
3,415,George Mburu Karanu,2000032902,18000,254722559933,31500.0
4,436,Eric Kiptoo Kosgei,2000033074,6000,254722859902,12000.0
5,439,David Kimutai Kirui,2000033024,6000,254721249078,12000.0
6,441,Esther Wambui Muya,2000033121,6000,254720269385,12000.0
7,450,Jane Wambui Kuria,2000032960,12000,254722633539,21000.0
8,462,Gordon Mbigo Nyagudi,2000033031,18000,254714606788,36000.0
9,463,Lucy Njoki Njogu,2000033205,12000,254722790605,24000.0


In [95]:
staff_account_limit_nov_2019[staff_account_limit_nov_2019.ACCOUNT.isnull()]

,Pfno,Name,ACCOUNT,Loan Limit,whizz_phone,new_limit_25_Dec


In [96]:
staff_account_limit.ACCOUNT_NUMBER.isnull().sum()

160

In [97]:
staff_account_limit_nov_2019

,Pfno,Name,ACCOUNT,Loan Limit,whizz_phone,new_limit_25_Dec
0,363,Lucy Wangui Ndungu,2000032981,6000,254722310293,10500.0
1,390,Rospa Jepngetich Kandie,2000033147,6000,254722466794,10500.0
2,403,Alice Wanjiru Weru,2000035405,6000,254722338270,7500.0
3,415,George Mburu Karanu,2000032902,18000,254722559933,31500.0
4,436,Eric Kiptoo Kosgei,2000033074,6000,254722859902,12000.0
5,439,David Kimutai Kirui,2000033024,6000,254721249078,12000.0
6,441,Esther Wambui Muya,2000033121,6000,254720269385,12000.0
7,450,Jane Wambui Kuria,2000032960,12000,254722633539,21000.0
8,462,Gordon Mbigo Nyagudi,2000033031,18000,254714606788,36000.0
9,463,Lucy Njoki Njogu,2000033205,12000,254722790605,24000.0


In [98]:
staff_account_limit.query('National_id == 24064275 ') #22197290   24991210   22716536

,Pfno,Name,ACCOUNT,MOBILE,Loan Limit,Type of whizz Account,cust_id,target saving,fixed saving,opening date,grade,income proxy,accountNumber,cbsCustomerID,customerID,customerNames,idNumber,mobileNumber,productCode,National_id,whizz_phone,ACCOUNT_NUMBER,frequency,recency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,adjuster,new_limit_25_Dec
61,1195,Benson Kennedy Irungu Karanja,2000081220,727551719,6000,STAFF SALARY ACCOUNT,117648.0,0.0,0.0,2010-04-30,03,10000.0,2.000081e+09,117648.0,47117.0,BENSON KARANJA,24064275,2.547276e+11,31111.0,24064275,254727551719,2.000081e+09,154.0,0.0,1173.0,4.0,3.0,3.0,433,2.0,12000.0


import psycopg2
t
    connection = psycopg2.connect(user = "datawarehouse",
                                  password = "P@ssw0rd",
                                  host = "128.2.4.15",
                                  port = "5432",
                                  database = "datawarehouse")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
            
            dbname = "datawarehouse",
                 host = "128.2.4.15", port = 5432,
                 user = "datawarehouse", password = pw)

import psycopg2
    connection = psycopg2.connect(user = "datawarehouse",
                                  password = "P@ssw0rd",
                                  host = "128.2.4.15",
                                  port = "5432",
                                  database = "datawarehouse")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")


In [103]:
loan_info = '''SELECT l.*,
 case when (CASE WHEN  DATE_CLOSED IS NULL THEN SYSDATE ELSE E.DATE_CLOSED  END)>L.ACC_EXP_DT then 'Default' else 'On time' end as status,
       CASE WHEN  DATE_CLOSED IS NULL THEN SYSDATE ELSE E.DATE_CLOSED  END as loan_close_date,
       (CASE WHEN  DATE_CLOSED IS NULL THEN SYSDATE ELSE E.DATE_CLOSED  END - L.ACC_EXP_DT) as Days_stayed_loan,
        l.ACC_OPEN_DT,to_char(CASE WHEN  DATE_CLOSED IS NULL THEN SYSDATE ELSE E.DATE_CLOSED  END-ACC_OPEN_DT) as Days_Stayed_with_loan,
CUST.SEX,
Oth.ID_NO,
to_char(round((SYSDATE-cust.DATE_OF_BIRTH)/365)) as AGE

FROM PROD.LNS_INSTANT_LOANS L
left join STAGING.W_EOM_ACCOUNT E on(E.ACCOUNT_NUMBER=L.LOAN_ACCOUNT)
left join PROD.OTHER_ID oth on (L.CUST_ID = oth.FK_CUSTOMERCUST_ID and oth.MAIN_FLAG = '1' )
left join PROD.CUSTOMER CUST on (CUST.CUST_ID = L.CUST_ID)
--FROM PROD.LNS_INSTANT_LOANS lns
where
trunc(L.TMSTAMP) >= '01-NOV-18'
and trunc(L.TMSTAMP) <= '31-DEC-19'
and L.MOBILE_TEL IN ('254722310293','254722466794','254722338270','254722559933','254722859902','254721249078','254720269385','254722633539','254714606788','254722790605','254722445823','254722506128','254722992269','254722232012','254722858740','254712603278','254728878117','254713054217','254722780800','254722306171','254722757293','254722520892','254722258597','254722831650','254722302365','254721829879','254723681065','254721750439','254721246093','254728077290','254728806687','254721246564','254710354925','254720862119','254722586861','254720865415','254720820054','254720478716','254724830153','254720700494','254724147213','254724291393','254722628054','254723456354','254701016677','254720777221','254720711124','254722394554','254723639144','254716216783','254724811282','254720259742','254721289875','254722932606','254728592319','254722222024','254712315535','254722369124','254726907634','254722991012','254722720060','254727551719','254724899710','254727175120','254721100295','254723394259','254726595752','254728071919','254722623336','254725501622','254722581540','254738905760','254721236122','254724274733','254720360443','254723334483','254725221432','254723383740','254724129816','254723482372','254729942258','254722826853','254719137994','254726998733','254725616377','254726540779','254727200743','254722332864','254725069276','254728068518','254700317499','254728216722','254724319721','254728039840','254720460923','254725815911','254725369659','254722705241','254712920859','254727337536','254720329647','254714402811','254726524275','254726359802','254728115798','254729340990','254712445933','254721703004','254720927052','254729498494','254721491324','254726930420','254715227650','254729035091','254722449643','254729340110','254725581520','254729833861','254721258123','254722392101','254724831069','254721236857','254722465748','254711922266','254720291470','254727796789','254717816467','254726543830','254721277333','254721686033','254722758354','254721426120','254721605385','254729485284','254732742271','254724315253','254721681281','254722477059','254726909476','254723282771','254710614827','254711175250','254713347128','254722214090','254713250705','254735911019','254711581765','254720385702','254722268758','254716749294','254722218960','254723446179','254723788794','254726472090','254716628304','254722269099','254714538554','254716023204','254725836265','254727744900','254724503789','254725630029','254723152851','254707492123','254721307899','254727291363','254720067288','254726293194','254721917152','254716286925','254712699832','254712816038','254714318327','254715862783','254713370626','254721452597','254722857259','254727241967','254727066314','254721278062','254710306435','254728814964','254715681200','254710323937','254711777893','254725161204','254727597342','254726455777','254725500056','254722426388','254725464237','254725690430','254720880342','254727560197','254721676613','254722608265','254726784200','254728105793','254722258004','254721207942','254726374365','254727461268','254718152062','254723680337','254725402891','254722333639','254722780090','254728935742','254721883439','254706551162','254711175089','254711525282','254716171086','254722156950','254723050484','254716315570','254723281297','254726466455','254724587924','254712349280','254722386984','254715030573','254720018494','254724718652','254726617753','254710760282','254728018749','254717596610','254723096985','254715684937','254717334392','254720070222','254725634581','254720802377','254726499099','254722626692','254711499852','254719572359','254727951127','254723016138','254721361834','254722352401','254721904654','254720935672','254724239611','254726126333','254721854162','254721443118','254722600172','254720264890','254722610274','254725791331','254723417304','254723845939','254723830315','254712449513','254722730071','254724890857','254700016072','254725549167','254726089016','254700304986','254723110436','254726206399','254724598283','254712758682','254701561835','254724282358','254707137333','254720805885','254721250373','254724985708','254717358019','254723491292','254700150693','254721554699','254720256476','254721416953','254725604868','254722885729','254729505957','254700012845','254720863868','254728658693','254711326395','254712806795','254733128157','254722397922','254706986082','254713326546','254724016691','254710194825','254723252774','254725025105','254720872039','254790705567','254746614239','254718099368','254706839192','254723022392','254700388366','254713296862','254706320903','254717745601','254717964803','254729487129','254715098322','254729216220','254719229760','254708948161','254719596589','254799541199','254713646923','254713994187','254780491114','254712879444','254790847868','254721645190','254716157271','254722895047','254716844363','254715243805','254726790781','254711409301','254720797994','254724215509','254742185842','254711147826','254729665465','254725300081','254724542852','254726838746','254713069852','254729471266','254726442656','254714009108','254700478128','254705902239','254700423062','254715197935','254702842523','254725837453','254714250961','254721137841','254704576375','254700399203','254727389342','254701763861','254705082062','254706581312','254725168343','254723215317','254721352802','254722783626','254716199057','254799054321','254720319899','254723594331','254710819074','254717048127','254729373548','254713901942','254780594331','254724252961','254741053087','254724044709','254727114520','254726704956','254724001457','254713603163','254722977981','254713605651','254721462728','254701628012','254718560895','254727725508','254701719486','254718942733','254723292173','254724796964','254711667511','254724629052','254725020420','254701597009','254714942229','254700215029','254715138204','254727775941','254798449650','254712773715','254795273366','254710768593','254718599476','254708766421','254790775864','254702392940','254705901004','254721419551','254713708608','254725418283','254716356788','254701125688','254790850705','254719897290','254715812674','254712932972','254722339409','254710789517','254713698117','254743235799','254726308069')
ORDER BY L.TMSTAMP DESC'''
loan_info_1 = pd.read_sql_query(loan_info, connection)
loan_info_1


,CUST_ID,LOAN_ACCOUNT,TMSTAMP,LOANS_PROGRAM_ID,PROGRAM_CURRENCY,MOBILE_TEL,LOAN_CD,AGREEMENT_ACCOUNT,AGREEMENT_CD,DEPOSIT_ACCOUNT,DEPOSIT_CD,C_DIGIT,INSTANT_LOAN_AMOUNT,INSTALL_FREQ,INSTALL_COUNT,ACC_OPEN_DT,ACC_EXP_DT,TRX_USER,BANKEMPLOYEE,ALLOCATION_TYPE,COLLATERAL_SN,APPLICATION_ID,LOAN_CAPITAL,LOAN_INTEREST,LOAN_EXPENCES,LOAN_COMMISSION,DEPOSIT_AMOUNT,TRX_COMMENTS,STATUS,LOAN_CLOSE_DATE,DAYS_STAYED_LOAN,ACC_OPEN_DT,DAYS_STAYED_WITH_LOAN,SEX,ID_NO,AGE
0,21571,0600214924,2019-12-20 12:18:28.304,1,22,254722992269,42,190000223420,37,2000033067,0,2,9000.0,0,0,2019-12-20,2020-01-18,KOCE0001,KOCE0001,1,0,214924,9000.0,270.00,0.0,648.0,8082.00,,On time,2019-12-20 18:17:37,-28.237766,2019-12-20,.762233796296296296296296296296296296296,M,11210826,50
1,193509,0600214906,2019-12-20 09:19:23.804,1,22,254727461268,46,190000223400,41,1935093201,0,2,4000.0,0,0,2019-12-20,2020-01-18,KOCE0001,KOCE0001,1,0,214906,4000.0,120.00,0.0,288.0,3592.00,,On time,2019-12-20 18:17:37,-28.237766,2019-12-20,.762233796296296296296296296296296296296,M,25954679,32
2,158122,0600214868,2019-12-19 16:58:05.226,1,22,254722465748,50,190000223362,45,7050000311,0,1,20000.0,0,0,2019-12-19,2020-01-17,KOCE0001,KOCE0001,1,0,214868,20000.0,600.00,0.0,1440.0,17960.00,,On time,2019-12-20 18:17:37,-27.237766,2019-12-19,1.7622337962962962962962962962962962963,F,21962036,40
3,146685,0600214810,2019-12-19 00:26:41.171,1,22,254726524275,83,190000223291,78,7050000246,0,9,2300.0,0,0,2019-12-19,2020-01-17,KOCE0001,KOCE0001,1,0,214810,2300.0,69.00,0.0,165.6,2065.40,,On time,2019-12-20 18:17:37,-27.237766,2019-12-19,1.7622337962962962962962962962962962963,F,26298503,31
4,146685,0600214810,2019-12-19 00:26:41.171,1,22,254726524275,83,190000223291,78,7050000246,0,9,2300.0,0,0,2019-12-19,2020-01-17,KOCE0001,KOCE0001,1,0,214810,2300.0,69.00,0.0,165.6,2065.40,,On time,2019-12-20 18:17:37,-27.237766,2019-12-19,1.7622337962962962962962962962962962963,F,26298503,31
5,190202,0600214807,2019-12-18 22:43:45.349,1,22,254725161204,52,190000223288,47,7050000475,0,1,1000.0,0,0,2019-12-19,2020-01-16,KOCE0001,KOCE0001,1,0,214807,1000.0,30.00,0.0,72.0,898.00,,On time,2019-12-19 00:00:00,-28.000000,2019-12-19,0,M,24015602,35
6,190202,0600214807,2019-12-18 22:43:45.349,1,22,254725161204,52,190000223288,47,7050000475,0,1,1000.0,0,0,2019-12-19,2020-01-16,KOCE0001,KOCE0001,1,0,214807,1000.0,30.00,0.0,72.0,898.00,,On time,2019-12-20 18:17:37,-26.237766,2019-12-19,1.7622337962962962962962962962962962963,M,24015602,35
7,198783,0600214759,2019-12-18 10:51:55.257,1,22,254726466455,15,190000223231,10,1987833101,0,0,9000.0,0,0,2019-12-18,2020-01-16,KOCE0001,KOCE0001,1,0,214759,9000.0,270.00,0.0,648.0,8082.00,,On time,2019-12-20 18:17:37,-26.237766,2019-12-18,2.7622337962962962962962962962962962963,M,25807919,32
8,198783,0600214759,2019-12-18 10:51:55.257,1,22,254726466455,15,190000223231,10,1987833101,0,0,9000.0,0,0,2019-12-18,2020-01-16,KOCE0001,KOCE0001,1,0,214759,9000.0,270.00,0.0,648.0,8082.00,,On time,2019-12-20 18:17:37,-26.237766,2019-12-18,2.7622337962962962962962962962962962963,M,25807919,32
9,115310,0600214738,2019-12-18 05:12:35.525,1,22,254722369124,84,190000223209,79,2000079432,0,8,3000.0,0,0,2019-12-18,2020-01-16,KOCE0001,KOCE0001,1,0,214738,3000.0,90.00,0.0,216.0,2694.00,,On time,2019-12-20 18:17:37,-26.237766,2019-12-18,2.7622337962962962962962962962962962963,F,12407927,47


In [105]:
loan_info_2 =loan_info_1

In [107]:
unique(loan_info_2.STATUS)

NameError: name 'unique' is not defined

In [54]:
export_csv = staff_account_limit_nov_2019.to_csv (r'/Users/allan.aswani/Documents/scorecardworks/staff_account_limit_dec_2019.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [35]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')